In [42]:
%matplotlib inline
%pylab inline
from time import sleep
import numpy as np 
import scipy
import os
import sys
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, confusion_matrix

from random import randint
from keras.callbacks import Callback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import LSTM
from keras.optimizers import RMSprop

from keras.layers.core import Dense, Dropout, Activation, Flatten


#Questions:
#question: Is there a better way to do the input than just flattening?

#Completed tasks
#Write a normal neural net to try to predict the next measurement with the window method in multiple dimensions (train_baseline_multi)
#Write a deep neural net to try to predict the next measurement with the window method in multiple dimensions (train_baseline_multi)
#Write a Random Forest Regressor to try to predict the next measurement with the window method in multiple dimensions (train_forest)
#Consolidate the pipeline - shape data / create models / validate models 
#Sliding window with a RNN to predict the next value (train_window_regression_rnn)
#Random Forest Classifier to classify FLAG at the next measurement(30 trees, 30 features)
#Neural Network Classifier (1 layer, 512 nodes) 
#Deep Neural Network Classifier (2 layers, 512 nodes) 
#Recurrent Neural Network for classification using window
#Complete all the quizzes and submit a request for MIMIC
#Sanity check for A = B
    #If it doesn't work, make sure the rolling/unrolling is correct
    #Switch order of patient and timestep
    #If it still doesn't work, try only feeding it one previous instead of 20 previous
#Completed today
#Fix the regression
    #Make it so that the matrix predictor gives 0 error
    #Make the other regressors make sense
    #Make all the regressions work
#Make all the classifications work similarly

#--------------------------------------------------------------------------------------------------------------------------------------------
#Regression
    #Write scaffolding for testing variation of len_sequence DONE
    #Fix the data generation DONE
    #Create the graphs
    #Write up Regression where A \neq B
    #Write up Regression where A = B
#Writeup: Classification 
    #Write scaffolding for testing variation of len_sequence 
    #Create the graphs
    #Write up Classification
#Sequence to sequence rnn classification
    #Get a 1-d version working DONE
    #Investigate what happens more closely by reducing the number of people to 2 DONE
    #Get a multiple dimension version working DONE
    #Go back to the toy example and make it work for larger dimensions
    #Bring it back to our synthetic data
#Figure out what we do if data is missing
#Implement it
#--------------------------------------------------------------------------------------------------------------------------------------------
#TODO 
#Investigate if any of our methods can actually predict in the time period where the flag is being flipped: e.g. the previous window has both 0 and 1
#Write a normal neural net to take prediction from rnn and predict 1/0
#Investigate this idea of deleting data
#Add a stateful try where the rnn is an autoencoder ( http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf) and then another LSTM

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [43]:
# fix random seed for reproducibility
np.random.seed(7)
V=np.random.randn(10)
print(V.dot(V), np.linalg.norm(V)**2)

8.33980975239 8.33980975239


In [44]:
#Setup for the generation of the data 
d=10 ##number of measurements per patient per time unit
k=2 ##complexity of the time series
n=200 ## number of patients
len_sequence = 10
sigmaw=0.2##randomness
T=175 ## amount of time measured for each patient (also batch size)
U=np.random.randn(d,k).dot(np.random.randn(k,d))
betashift=np.random.randn(d)/np.sqrt(d)  ## used to check when we switch to the other time series
#normalized U
A=0.9*U/np.linalg.norm(U,ord=2)
#V is also d by d
V=np.random.randn(d,k).dot(np.random.randn(k,d))
#normalized V
B = .9*V/np.linalg.norm(V,ord=2) 
#B = A #Set B = A for sanity checking
flag = 1

In [45]:
def generate_data():
    X = np.zeros((n,T,d)) #this represents n patients, in d-dimensions, for T time-steps
    flags = np.zeros((n,T))
    #Generate the data, fill in X, flags
    for j in range(n): #for each patient
        X[j,0,:]=np.random.randn(d)/np.sqrt(d)
        flag=1
        for i in range(1,T): #for each time      
            if flag and ((X[j,i-1,:].dot(betashift))>.5): ##if flag was true and we go above a certain threshold, then set flag=0
                flag = 0
            ## we can think of flag has flag==1 means not critical, flag==0 means critical
            if flag:
                X[j,i,:]=A.dot(X[j,i-1,:])+sigmaw*np.random.randn(d)
                flags[j,i] = 0
            else:
                X[j,i,:]=B.dot(X[j,i-1,:])+sigmaw*np.random.randn(d)
                flags[j, i] = 1
    return(X, flags)

In [46]:
def generate_crazy(len_sequence, X, flags):
    ## X is a n by d by T tensorx
    ## n represents independent trials (think of them as patients or time periods for patients separated)
    ## T represents the time horizon (how long we observe a patient, for now T is the same for everybody, but we can imagine that changing)
    ## d is the number of measurements that we have per patient per time.
    ## flags are num_patients * num_timesteps * dimensionality
    crazy = np.array([X[:, i:i + len_sequence] for i in range(X.shape[1] - len_sequence)])
    crazy_y = np.array([X[:, i + len_sequence] for i in range(X.shape[1] - len_sequence)])
    crazy_flags = np.array([flags[:, i + len_sequence] for i in range(X.shape[1] - len_sequence)])
    return (crazy, crazy_y, crazy_flags)

In [47]:
print(X.shape)
print(flags.shape)
print(crazy.shape)
print(crazy_y.shape)
print(crazy_flags.shape)

#print(flags[6])
#print(crazy_flags[:,6])
#print(X[0,:,0])
#print(crazy[:,0,:,0])
#print(crazy_y[:,0,0])

(200, 175, 10)
(200, 175)
(158, 200, 17, 10)
(158, 200, 10)
(158, 200)


In [ ]:
#THIS ONE IS FOR REGRESSION
all_models = []
all_data = []
rnn_data = []
(X, flags) = generate_data()

for i in range(1,20,4):
    len_sequence = i
    (crazy, crazy_y, crazy_flags) = generate_crazy(len_sequence, X, flags)
    (X_train, y_train, X_test, y_test) = prepare_baseline(crazy, crazy_y)
    all_data.append((X_train, y_train, X_test, y_test))
    
    models = {}
    tree_models = train_forest(X_train, y_train)
    models['tree'] = tree_models[0]
    models['nn'] = train_baseline_nn(X_train, y_train, X_test, y_test)
    models['deep_nn'] = train_baseline_multi_deep(X_train, y_train, X_test, y_test)
    
    (X_train, y_train, X_test, y_test) = prepare_rnn(crazy, crazy_y)
    rnn_data.append((X_train, y_train, X_test, y_test))
    
    models['window_rnn'] = train_window_regression_rnn(X_train, y_train, X_test, y_test)
    
    all_models.append(models)

In [ ]:
#THIS ONE IS FOR CLASSIFICATION
all_models = []
all_data = []
rnn_data = []
(X, flags) = generate_data()

for i in range(1,20,4):
    len_sequence = i
    (crazy, crazy_y, crazy_flags) = generate_crazy(len_sequence, X, flags)
    (X_train, y_train, X_test, y_test) = prepare_classification_baseline(crazy, crazy_flags)
    all_data.append((X_train, y_train, X_test, y_test))
    
    #(X_train, y_train, X_test, y_test) = prepare_baseline()
    models = {}
    tree_models = train_classification_forest(X_train, y_train)
    models['tree'] = tree_models[0]
    models['nn'] = train_window_classification_nn(X_train, y_train, X_test, y_test)
    models['deep_nn'] = train_window_classification_deep_nn(X_train, y_train, X_test, y_test)

    (X_train, y_train, X_test, y_test) = prepare_classification_rnn(crazy, crazy_flags)
    rnn_data.append((X_train, y_train, X_test, y_test))
    
    models['window_rnn'] = train_window_classification_rnn(X_train, y_train, X_test, y_test)
    
    all_models.append(models)

(174, 200, 1, 10)
(174, 200)
(34800, 10)
(34800,)
(27840, 10)
(27840,)
(6960, 10)
(6960,)
building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    5.9s finished



Train on 27840 samples, validate on 6960 samples
Epoch 1/10
27840/27840 [==============================] - 2s - loss: 0.5573 - acc: 0.7422 - val_loss: 0.4956 - val_acc: 0.7922
Epoch 2/10
27840/27840 [==============================] - 2s - loss: 0.5363 - acc: 0.7521 - val_loss: 0.4904 - val_acc: 0.7938
Epoch 3/10
27840/27840 [==============================] - 2s - loss: 0.5332 - acc: 0.7528 - val_loss: 0.4917 - val_acc: 0.7918
Epoch 4/10
27840/27840 [==============================] - 2s - loss: 0.5320 - acc: 0.7547 - val_loss: 0.4961 - val_acc: 0.7899
Epoch 5/10
27840/27840 [==============================] - 2s - loss: 0.5311 - acc: 0.7543 - val_loss: 0.4923 - val_acc: 0.7922
Epoch 6/10
27840/27840 [==============================] - 2s - loss: 0.5307 - acc: 0.7541 - val_loss: 0.4914 - val_acc: 0.7937
Epoch 7/10
27840/27840 [==============================] - 2s - loss: 0.5304 - acc: 0.7543 - val_loss: 0.4905 - val_acc: 0.7925
Epoch 8/10
27840/27840 [==============================] - 3s 

In [ ]:
counter = 1
for (model, data, rnn_datum) in zip(all_models, all_data, rnn_data):
    (X_train, y_train, X_test, y_test) = data
    print("tree for sequence length " + str(counter) + '\n')
    analyze(model['tree'], X_test, y_test)
    print('\n')
    print("neural net for sequence length " + str(counter) + '\n')
    analyze(model['nn'], X_test, y_test)
    print('\n')
    print("deep neural net for sequence length " + str(counter) + '\n')
    analyze(model['deep_nn'], X_test, y_test)
    print('\n')
    (X_train, y_train, X_test, y_test) = rnn_datum
    print("window recurrent neural net for sequence length " + str(counter) + '\n')
    analyze(model['window_rnn'], X_test, y_test, rnn=True)
    print('\n')    
    counter += 1

In [ ]:
#Analysis of Regression when A = B
#all_zero error: 0.168895379401
#smart_predictor error: 0.159935136096

sequence_lengths = [1,5,9,13,17]
trees = [0.169523201816,0.163457677349,0.163639987636,0.163690879409,0.163651731491 ]
nns = [0.160142227167,0.161104080877,0.161564311629,0.161922431239,0.162239372943]
deep_nns = [0.160053838298,0.160528898159,0.160786978029,0.160851565687,0.160993068655]
window_rnns = [0.160126886333,0.160145655568,0.160189070326,0.160301765207,0.159979021872]

fig = plt.figure()
plt.plot(sequence_lengths, trees, 'r', label='Random Forest')
plt.plot(sequence_lengths, nns, 'b', label='Neural Network')
plt.plot(sequence_lengths, deep_nns, 'y', label='Deep Neural Network')
plt.plot(sequence_lengths, window_rnns, 'g', label='Recurrent Neural Network')
plt.ylabel('Mean Squared Error')
plt.xlabel('Number of measurements per training sample')
plt.title('Effect of window size on MSE for regression models where A = B')
plt.legend()
plt.savefig('len_sequence_v_MSE_A=B')

In [ ]:
#Analysis of Regression when A \neq B
# Using just Ax_{t-1} as a predictor 0.163159100363
# Using just 0 as a predictor: 0.168873889664
sequence_lengths = [1,5,9,13,17]
trees = [0.172481252728,  0.165948494495, 0.166031396918, 0.16612096304, 0.166453330425]
nns = [0.162772912922,0.163784221657, 0.164316503743, 0.164711745367, 0.165270914672]
deep_nns = [0.162570283671,0.162980507276, 0.163516877129, 0.163565435967, 0.163733663807]
window_rnns = [0.162826821642, .16191162646, 0.161616112766, 0.161263587408, 0.161016470274]

fig = plt.figure()
plt.plot(sequence_lengths, trees, 'r', label='Random Forest')
plt.plot(sequence_lengths, nns, 'b', label='Neural Network')
plt.plot(sequence_lengths, deep_nns, 'y', label='Deep Neural Network')
plt.plot(sequence_lengths, window_rnns, 'g', label='Recurrent Neural Network')
plt.ylabel('Mean Squared Error')
plt.xlabel('Number of measurements per training sample')
plt.title('Effect of window size on MSE for regression models')
plt.legend()
plt.savefig('len_sequence_v_MSE')

In [ ]:
#Given a model that uses the flattened window, this will print its mean squared error and plot it's results versus reality
def analyze(model, X_test, y_test, rnn=False):
    plt.clf()
    results = model.predict(X_test)
    manual = np.zeros(y_test.shape)
    if not rnn:
        for i in range(manual.shape[0]):
            manual[i,:] = A.dot(X_test[i,-d:])
    else:
        for i in range(manual.shape[0]):
            manual[i,:] = A.dot(X_test[i,-1,:])
    p_num=1
    preds = results[p_num*T:(p_num+1)*T,:]
    reals = y_test[p_num*T:(p_num+1)*T,:]
    #manuals = manual[int(manual.shape[0]*.5):(int(manual.shape[0]*.5) + 10), :]
    
    plt.plot(preds, color='red')
    plt.plot(reals, color='blue')
    plt.ylabel('Measurements')
    plt.xlabel('Time')
    
    #plt.title('Regression with Random Forest')
    #plt.savefig('tree_regression_projection_a_eq_b')
    
    #plt.title('Regression with Neural Network')
    #plt.savefig('nn_regression_projection_a_eq_b')
    
    #plt.title('Regression with Deep Neural Network')
    #plt.savefig('deep_nn_regression_projection_a_eq_b')
    
    plt.title('Regression with Recurrent Neural Network')
    plt.savefig('window_rnn_regression_projection_a_eq_b')
    
    #plt.plot(manuals, color='yellow')
    
    all_z = np.zeros(y_test.shape)
    
    print("all_zero error: " + str(mean_absolute_error(y_test, all_z)))
    print("smart_predictor error: " + str(mean_absolute_error(y_test, manual)))
    print("Actual Error: " + str(mean_absolute_error(y_test, results)))

In [ ]:
#(X_train, y_train, X_test, y_test) = all_data[1]
#analyze(all_models[1]['tree'], X_test, y_test)

#(X_train, y_train, X_test, y_test) = all_data[0]
#analyze(all_models[0]['nn'], X_test, y_test)
#analyze(all_models[0]['deep_nn'], X_test, y_test)

#(X_train, y_train, X_test, y_test) = rnn_data[4]
#analyze(all_models[4]['window_rnn'], X_test, y_test, rnn=True)


In [ ]:
pylab.rcParams['figure.figsize'] = (17, 12)

In [15]:
def prepare_baseline(crazy, crazy_y):
    #Reshape data and flatten for baseline models
    rolled_x = crazy[:,:,:,:]
    rolled_y = crazy_y[:,:,:]

    print (rolled_x.shape)
    print (rolled_y.shape)

    unrolled_x = np.zeros((crazy.shape[0]*crazy.shape[1], crazy.shape[2] * crazy.shape[3]))
    unrolled_y = np.zeros((crazy.shape[0] * crazy.shape[1], crazy.shape[3]))

    for i in range(crazy.shape[1]): #patients
        for j in range(crazy.shape[0]): #time steps
            unrolled_y[i * crazy.shape[0] + j] = rolled_y[j,i,:]            
            for l in range(crazy.shape[2]):#len_sequence
                for k in range(crazy.shape[3]): #dimensionality
                    unrolled_x[i * crazy.shape[0] + j][l*crazy.shape[3] + k] = rolled_x[j,i,l,k]

    print (unrolled_x.shape)
    print (unrolled_y.shape)
    transition_point = int(unrolled_x.shape[0] * .8)
    
    X_train = np.array(unrolled_x[:transition_point,:])
    y_train = np.array(unrolled_y[:transition_point])
    X_test = np.array(unrolled_x[transition_point:,:])
    y_test = np.array(unrolled_y[transition_point:])

    print (X_train.shape)
    print (y_train.shape)
    print (X_test.shape)
    print (y_test.shape)
    
    return (X_train, y_train, X_test, y_test)

In [16]:
def prepare_classification_baseline(crazy, crazy_flags):
    #Reshape data and flatten for baseline models
    rolled_x = crazy[:,:,:,:]
    rolled_y = crazy_flags[:,:]

    print (rolled_x.shape)
    print (rolled_y.shape)
    unrolled_x = np.zeros((crazy.shape[0]*crazy.shape[1], crazy.shape[2] * crazy.shape[3]))
    unrolled_y = np.zeros((crazy.shape[0] * crazy.shape[1]))

    for i in range(crazy.shape[1]): #patients
        for j in range(crazy.shape[0]): #time steps
            unrolled_y[i * crazy.shape[0] + j] = rolled_y[j,i]
            for l in range(crazy.shape[2]):#len_sequence
                for k in range(crazy.shape[3]): #dimensionality
                    unrolled_x[i * crazy.shape[0] + j][l*crazy.shape[3] + k] = rolled_x[j,i,l,k]

    print (unrolled_x.shape)
    print (unrolled_y.shape)
    transition_point = int(unrolled_x.shape[0] * .8)
    
    X_train = np.array(unrolled_x[:transition_point,:])
    y_train = np.array(unrolled_y[:transition_point])
    X_test = np.array(unrolled_x[transition_point:,:])
    y_test = np.array(unrolled_y[transition_point:])
    
    print (X_train.shape)
    print (y_train.shape)
    print (X_test.shape)
    print (y_test.shape)
    
    return (X_train, y_train, X_test, y_test)

In [17]:
#Given a model that uses the flattened window for classification,this will print its confusion matrix and plot it's results versus reality
def analyze_classification(model, X_test, y_test):
    results = model.predict(X_test)
    #only plotting the first 400 because otherwise you can't see anything
    preds = results[:400]
    reals = y_test[:400]
    plt.plot(preds, color='red')
    plt.plot(reals, color='blue')
    print(confusion_matrix(y_test, results))
    #Compare to all zeros
    """
    print("Try all zeros")
    zeros = np.zeros(5600)
    print(confusion_matrix(y_test, zeros))
    """

In [18]:
def train_forest(X_train, y_train):
    ne = [min(d * len_sequence, 40)]
    nf = [min(d * len_sequence, 40)]
    models = []
    for e in ne:
        for f in nf:   
            model = RandomForestRegressor(n_estimators=e, max_features = f, verbose=2)
            model.fit(X_train, y_train)
            models.append(model)
    return models

In [19]:
def train_baseline_nn(X_train, y_train, X_test, y_test):
    #Baseline model in multiple dimensions: predicts the next measurement, all 10 dimensions
    rms = RMSprop()
    model = Sequential()
    model.add(Dense(512, input_shape=(X_train.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(y_train.shape[1]))
    model.compile(loss='mean_squared_error', optimizer=rms)
    model.fit(X_train, y_train, batch_size=32, nb_epoch=10, verbose=1,
              validation_data=(X_test, y_test))
    return model

In [20]:
def train_baseline_multi_deep(X_train, y_train, X_test, y_test):
    #Baseline model in multiple dimensions: predicts the next measurement, all 10 dimensions
    rms = RMSprop()
    model = Sequential()
    model.add(Dense(512, input_shape=(X_train.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(y_train.shape[1]))
    model.compile(loss='mean_squared_error', optimizer=rms)
    model.fit(X_train, y_train, batch_size=32, nb_epoch=10, verbose=1,
              validation_data=(X_test, y_test))
    return model

In [21]:
def prepare_rnn(crazy, crazy_y):
    #Reshape data and flatten for baseline models
    rolled_x = crazy[:,:,:,:]
    rolled_y = crazy_y[:,:,:]

    print (rolled_x.shape)
    print (rolled_y.shape)

    unrolled_x = np.zeros((crazy.shape[0] * crazy.shape[1], crazy.shape[2], crazy.shape[3]))
    unrolled_y = np.zeros((crazy.shape[0] * crazy.shape[1], crazy.shape[3]))

    #maybe we should switch the order of these (but also maybe not it depends how batching works)
    for i in range(crazy.shape[1]): #patients
        for j in range(crazy.shape[0]): #time steps
            unrolled_x[i * crazy.shape[0] + j] = rolled_x[j,i,:,:]
            unrolled_y[i * crazy.shape[0] + j] = rolled_y[j,i,:]            

    print (unrolled_x.shape)
    print (unrolled_y.shape)

    transition_point = int(unrolled_x.shape[0] * .8)
    
    X_train = np.array(unrolled_x[:transition_point,:])
    y_train = np.array(unrolled_y[:transition_point])
    X_test = np.array(unrolled_x[transition_point:,:])
    y_test = np.array(unrolled_y[transition_point:])
    
    print (X_train.shape)
    print (y_train.shape)
    print (X_test.shape)
    print (y_test.shape)
    
    return (X_train, y_train, X_test, y_test)
    

In [22]:
#Predict the next measurement.  Trained using the window method.  A GRU + a Dense layer
def train_window_regression_rnn(X_train, y_train, X_test, y_test):
    model = Sequential()
    model.add(GRU(16,activation='relu', input_dim = d))
    model.add(Dense(128))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))
    model.add(Dense(d))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X_train, y_train, batch_size=32, nb_epoch=10, verbose=1,
              validation_data=(X_test, y_test))
    return model

In [23]:
def prepare_classification_rnn(crazy, crazy_flags):
    #Reshape data and flatten for baseline models
    rolled_x = crazy[:,:,:,:]
    rolled_y = crazy_flags[:,:]

    print (rolled_x.shape)
    print (rolled_y.shape)

    unrolled_x = np.zeros((crazy.shape[0] * crazy.shape[1], crazy.shape[2], crazy.shape[3]))
    unrolled_y = np.zeros((crazy.shape[0] * crazy.shape[1]))

    #maybe we should switch the order of these (but also maybe not it depends how batching works)
    for i in range(crazy.shape[1]): #time steps
        for j in range(crazy.shape[0]): #patients
            unrolled_x[i * crazy.shape[0] + j] = rolled_x[j,i,:,:]
            unrolled_y[i * crazy.shape[0] + j] = rolled_y[j,i]            

    print (unrolled_x.shape)
    print (unrolled_y.shape)
    transition_point = int(unrolled_x.shape[0] * .8)
    
    X_train = np.array(unrolled_x[:transition_point,:])
    y_train = np.array(unrolled_y[:transition_point])
    X_test = np.array(unrolled_x[transition_point:,:])
    y_test = np.array(unrolled_y[transition_point:])

    print (X_train.shape)
    print (y_train.shape)
    print (X_test.shape)
    print (y_test.shape)
    
    return (X_train, y_train, X_test, y_test)


In [31]:
def train_classification_forest(X_train, y_train):
    ne = [min(d * len_sequence, 40)]
    nf = [min(d * len_sequence, 40)]
    models = []
    for e in ne:
        for f in nf:   
            model = RandomForestClassifier(n_estimators=e, max_features = f, verbose=2)
            model.fit(X_train, y_train)
            models.append(model)
    return models

In [ ]:
(X_train, y_train, X_test, y_test) = prepare_classification_baseline()

In [ ]:
models = train_classification_forest(X_train, y_train)

In [ ]:
for m in models: 
    print ("num_estimators: " + str(m.n_estimators))
    print ("Max Features: " + str(m.max_features))
    analyze_classification(m, X_test, y_test)

In [25]:
def train_window_classification_nn(X_train, y_train, X_test, y_test):
    #Baseline model in multiple dimensions: predicts the next measurement, all 10 dimensions
    rms = RMSprop()
    model = Sequential()
    model.add(Dense(512, input_shape=(X_train.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=32, nb_epoch=10, verbose=1,
              validation_data=(X_test, y_test))    
    return model

In [ ]:
model = train_window_classification_nn(X_train, y_train, X_test, y_test)

In [ ]:
results = model.predict(X_test)[:,0]
results = np.where(results>.5, 1, 0)
print (results)
#only plotting the first 400 because otherwise you can't see anything
preds = results[:400]
reals = y_test[:400]

plt.plot(preds, color='red')
plt.plot(reals, color='blue')
print(confusion_matrix(y_test, results))
#Compare to all zeros

In [33]:
def train_window_classification_deep_nn(X_train, y_train, X_test, y_test):
    #Baseline model in multiple dimensions: predicts the next measurement, all 10 dimensions
    rms = RMSprop()
    model = Sequential()
    model.add(Dense(512, input_shape=(X_train.shape[1],)))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=32, nb_epoch=10, verbose=1,
              validation_data=(X_test, y_test))    
    return model

In [ ]:
model2 = train_window_classification_deep_nn(X_train, y_train, X_test, y_test)

In [ ]:
results = model2.predict(X_test)[:,0]
results = np.where(results>.5, 1, 0)
print (results)
#only plotting the first 400 because otherwise you can't see anything
preds = results[:400]
reals = y_test[:400]

plt.plot(preds, color='red')
plt.plot(reals, color='blue')
print(confusion_matrix(y_test, results))
#Compare to all zeros

In [34]:
#Predict the next measurement.  Trained using the window method.  A GRU + a Dense layer
def train_window_classification_rnn(X_train, y_train, X_test, y_test):
    model = Sequential()
    model.add(GRU(16,activation='relu', input_dim = d))
    model.add(Dense(128))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.fit(X_train, y_train, batch_size=32, nb_epoch=10, verbose=1,
              validation_data=(X_test, y_test))
    return model

In [ ]:
(X_train, y_train, X_test, y_test) = prepare_classification_rnn()

In [ ]:
model = train_window_classification_rnn(X_train, y_train, X_test, y_test)

In [ ]:
results = model.predict(X_test)[:,0]
results = np.where(results>.5, 1, 0)
print (results)
#only plotting the first 400 because otherwise you can't see anything
preds = results[:400]
reals = y_test[:400]

plt.plot(preds, color='red')
plt.plot(reals, color='blue')
print(confusion_matrix(y_test, results))
#Compare to all zeros

In [ ]:
print(X.shape)
updated_flags = flags[:,:,0]
print (updated_flags.shape)
new_x = np.zeros((X.shape[0] * X.shape[1], X.shape[2],1))
for i in range(X.shape[0]):
    for ii in range(X.shape[1]):
        for iii in range(X.shape[2]):
            new_x[i * X.shape[1] + ii][iii][0] = X[i][ii][iii]
new_flags = np.zeros((X.shape[0] * X.shape[1], 1))
for i in range(updated_flags.shape[0]):
    for ii in range(updated_flags.shape[1]):
        new_flags[i * new_flags.shape[1] + ii][0] = updated_flags[i][ii]

transition = 50
X_train = new_x[0: transition * X.shape[1]]
X_test = new_x[transition * X.shape[1]:2*transition*X.shape[1]]
y_train = new_flags[0:transition*X.shape[1]]
y_test = new_flags[transition*X.shape[1]:2*transition*X.shape[1]]

In [ ]:
updated_flags[2,:]

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
class ResetStatesCallback(Callback):
    def __init__(self):
        self.counter = 0

    def on_batch_begin(self, batch, logs={}):
        if self.counter % T == 0:
            self.model.reset_states()
        self.counter += 1

In [ ]:
# STATEFUL MODEL
print('Build STATEFUL model...')
model = Sequential()
model.add(LSTM(512,
               batch_input_shape=(1, d, 1), return_sequences=False,
               stateful=True))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
          callbacks=[ResetStatesCallback()],
      batch_size=1, nb_epoch=1,
          shuffle=False, validation_data=(X_test, y_test))


In [ ]:
results = model.predict(X_test, batch_size=1, verbose=1)

In [ ]:
probs = results

In [ ]:
plt.plot(y_train, color='red')

In [ ]:
results = probs[:,0]
results = np.where(results>.5, 1, 0)

In [ ]:
rando_sum = 0
for i in range(8750):
    rando_sum += results[i]
print(rando_sum)

In [ ]:
results = model.predict(X_test)[:,0]
results = np.where(results>.5, 1, 0)
print (results)
#only plotting the first 400 because otherwise you can't see anything
preds = results[:400]
reals = y_test[:400]

plt.plot(preds, color='red')
plt.plot(reals, color='blue')
print(confusion_matrix(y_test, results))
#Compare to all zeros

In [ ]:
#EVERYTHING BELOW HERE IS NOT WORKING / OLD CODE

In [ ]:
#given train_split, train the model using the first train_split examples
#which = 0 for the normal prediction
#which = 1 for crazy_flags
#which = 2 for oned_flags
def train_stage1(train_split, which):
    #First model: Using the sliding window
    hists = []
    model = Sequential()
    model.add(GRU(128,activation='relu', input_shape=(crazy.shape[2:])))
    if which == 2:
        model.add(Dense(1))
    else:
        model.add(Dense(d))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(train_split): # for each patient
        this_patient_x = crazy[:,i,:,:]
        if which == 0:
            this_patient_y = crazy_y[:,i,:]
        elif which == 1:
            this_patient_y = crazy_flags[:,i,:]
        else:
            this_patient_y = oned_flags[:,i,0:1]
        for ii in range(crazy.shape[0]): #for each window of time sequence
            this_patient_x_i = np.array([this_patient_x[ii,:,:]])
            if which == 2:
                this_patient_y_i = np.array([this_patient_y[ii,0:1]])
            else:
                this_patient_y_i = np.array([this_patient_y[ii,:]])
            hists.append(model.train_on_batch(this_patient_x_i, this_patient_y_i))
        model.reset_states()
    return hists, model

In [ ]:
def train_stage2(train_split):
    #second model: trading a stateful rnn using this batch thing
    hists = []
    model = Sequential()
    
    X  # this is our input data, of shape (32, 21, 16)
    # we will feed it to our model in sequences of length 10

    model = Sequential()
    model.add(LSTM(32, batch_input_shape=(32, 10, 16), stateful=True))
    model.add(Dense(16, activation='softmax'))

    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

    # we train the network to predict the 11th timestep given the first 10:
    model.train_on_batch(X[:, :10, :], np.reshape(X[:, 10, :], (32, 16)))

    # the state of the network has changed. We can feed the follow-up sequences:
    model.train_on_batch(X[:, 10:20, :], np.reshape(X[:, 20, :], (32, 16)))

    # let's reset the states of the LSTM layer:
    model.reset_states()

    # another way to do it in this case:
    model.layers[0].reset_states()
    
    model.add(GRU(128,activation='relu', input_shape=(crazy.shape[2:])))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(train_split): # for each patient
        this_patient_x = crazy[:,i,:,:]
        if which == 0:
            this_patient_y = crazy_y[:,i,:]
        elif which == 1:
            this_patient_y = crazy_flags[:,i,:]
        else:
            this_patient_y = oned_flags[:,i,0:1]
        for ii in range(crazy.shape[0]): #for each window of time sequence
            this_patient_x_i = np.array([this_patient_x[ii,:,:]])
            if which == 2:
                this_patient_y_i = np.array([this_patient_y[ii,0:1]])
            else:
                this_patient_y_i = np.array([this_patient_y[ii,:]])
            hists.append(model.train_on_batch(this_patient_x_i, this_patient_y_i))
        model.reset_states()
    return hists, model

In [ ]:
oned_flags.shape
(hists, model) = train_stage1(10, 2)

In [ ]:
#compare the y between them
print(oned_flags[:, 0, 0:1].shape)
#this is y for normal stuff
print(crazy_y[:,i,:][0,:].shape)
#this is y for the newest one
print((oned_flags[:,0,0:1][0,0:1]).shape)
#compare the x between them


In [ ]:
print(crazy[:, patient_num, :, :].shape)
print((crazy[:,0,:,:][0,:,:]).shape)

crazy[:,patient_num,:,:]

In [ ]:
#evaluate the model on the last 10 samples
#which = 0 for the normal prediction
#which = 1 for crazy_flags
#which = 2 for oned_flags
def evaluate(model, which):
    evaluations = []
    for i in range(n-10, n): # for each patient not in the training
        this_patient_x = crazy[:,i,:,:]
        if which == 0:
            this_patient_y = crazy_y[:,i,:]
        elif which == 1:
            this_patient_y = crazy_flags[:,i,:]
        else:
            this_patient_y = oned_flags[:,i,:]
        for ii in range(crazy.shape[0]): #for each window of time sequence
            this_patient_x_i = np.array([this_patient_x[ii,:,:]])            
            this_patient_y_i = np.array([this_patient_y[ii,:]])
            evaluations.append(model.evaluate(this_patient_x_i, this_patient_y_i, verbose=0))
        model.reset_states()
    return evaluations

In [ ]:
histories = {}
all_evals = {}
which = 2
for training_amount in range(10, 100, 10):
    #Train the model
    (hists, model) = train_stage1(training_amount, which)

    #Check training results of the data
    hist_values = []
    for hist in hists:
        hist_values.append(hist.item())
    #plt.plot(hist_values)
    histories[training_amount] = (sum(hist_values)/float(len(hist_values)))

    #Check evaluation of the model
    evaluations = evaluate(model, which)
    #plt.plot(evaluations)
    all_evals[training_amount] = (sum(evaluations)/float(len(evaluations)))

In [ ]:
print(histories)
for key in histories:
    print (key)
    print (histories[key] - all_evals[key])
print(all_evals)

In [ ]:
#Visually check the prediction of the first reading versus what's actually happening
#(178, 200, 20, 10)

#choose a patient
patient_num = randint(101,199)
#choose a measuremen
measurement = 0

#Get what we are aiming for
if which == 0:
    reality = crazy_y[:, patient_num, measurement]
elif which == 1:
    reality = crazy_flags[:, patient_num, measurement]
elif which == 2:
    reality = oned_flags[:, patient_num, 0:1]
#Get the predictions
model.reset_states()
predictions = model.predict(crazy[:,patient_num,:,:])

#Plot prediction versus reality
plt.plot(predictions[:,0], color='red')
plt.plot(reality, color='blue')

In [ ]:
#Compute MSE
dif = reality - predictions[:,0]
dif = dif ** 2
print ("MSE:")
print (sum(dif)/178)

#Plot the difference
plt.plot(dif)